In [12]:
import pandas as pd
import numpy as np
# Load the data
data = pd.read_csv('TEST')
# data = pd.read_csv('events_HK_processed_18.csv')
# data = pd.read_csv('events_HK_2015_2020_test.csv')
# Display the first few rows of the dataframe
data.head()

,country_code_encoded_index,SQLDATE,EventCode,QuadClass,GoldsteinScale,AvgTone,NumArticles,NumMentions,Contains14sub,Contains14,country_code_encoded,Year,Month,DayOfMonth
0,0,2015-02-19,141,3,-6.5,-3.543307,1,1,1,0,0,2015,2,19
1,0,2015-02-20,141,3,-6.5,-3.543307,1,1,1,0,0,2015,2,20
2,0,2015-02-21,141,3,-6.5,-2.558272,10,10,1,0,0,2015,2,21
3,0,2015-02-22,141,3,-6.5,-8.896797,10,10,1,0,0,2015,2,22
4,0,2015-02-23,141,3,-6.5,-8.160237,10,10,1,0,0,2015,2,23


In [13]:
# data = data.drop(columns=['Contains14sub', 'ActionGeo_CountryCode', 'EventCode'])
# data.rename(columns={'Contains14': 'lag_fut_Contains14 1'}, inplace=True)
# data['AvgTone'].value_counts()

In [14]:
data = data.drop(columns=['Contains14sub', 'EventCode', 'country_code_encoded_index', 'country_code_encoded', 'Year', 'Month', 'DayOfMonth', 'SQLDATE'])
data.rename(columns={'Contains14': 'lag_fut_Contains14 1'}, inplace=True)

In [15]:
data.value_counts('lag_fut_Contains14 1')

lag_fut_Contains14 1
0    1720
1      57
dtype: int64

In [16]:
#  define a window in the future to predict
Days_in_future = 2

In [17]:


target_name = 'will_be_1_anytime_next_'+str(Days_in_future)+'_days'
# Create a rolling window of 7 days and check if there's any 1 in 'lag_fut_Contains14 1' column within this window
data[target_name] = data['lag_fut_Contains14 1'].rolling(window=Days_in_future, min_periods=1).apply(lambda x: 1 if x.sum() > 0 else 0)

# Shift the 'will_be_1_anytime_next_week' column up by 1 place
data[target_name] = data[target_name].shift(-1)

# Replace NaN values with 0
# data[target_name].fillna(0, inplace=True)
data.dropna(inplace=True)


# Convert the new column to integers
data[target_name] = data[target_name].astype(int)

# Display the first few rows of the dataframe
data.head(10)

,SQLDATE,QuadClass,GoldsteinScale,AvgTone,NumArticles,NumMentions,lag_fut_Contains14 1,Year,Month,DayOfMonth,will_be_1_anytime_next_2_days
0,2015-02-19,3,-6.5,-3.543307,1,1,0,2015,2,19,0
1,2015-02-20,3,-6.5,-3.543307,1,1,0,2015,2,20,0
2,2015-02-21,3,-6.5,-2.558272,10,10,0,2015,2,21,0
3,2015-02-22,3,-6.5,-8.896797,10,10,0,2015,2,22,0
4,2015-02-23,3,-6.5,-8.160237,10,10,0,2015,2,23,0
5,2015-02-24,3,-6.5,0.471698,8,8,0,2015,2,24,0
6,2015-02-25,3,-6.5,0.155521,10,10,0,2015,2,25,0
7,2015-02-26,3,-6.5,-0.838574,2,2,0,2015,2,26,0
8,2015-02-27,3,-6.5,-9.756098,3,3,0,2015,2,27,0
9,2015-02-28,3,-6.5,-9.756098,3,3,0,2015,2,28,0


In [18]:
# plt.scatter(data.index,data['lag_fut_Contains14 1'])

In [19]:
# Define lookback and buffer window
#  Units are number of days
window_gap = 0
lookback = 10

In [20]:
def rolling(df, lookback_lag_target, roll_range):
    df_lag = df.copy()
    # Create lagged features
    for lag in lookback_lag_target:
        df_lag[f'lag_{lag}'] = df_lag[target_name].shift(lag)
        # df[f'lagGoldstein_{lag}'] = df['GoldsteinScale'].shift(lag)
        # df[f'lagAvgTone_{lag}'] = df['AvgTone'].shift(lag)
        # df[f'lagAvgTone_{9}'] = df['AvgTone'].shift(9)
        # df[f'lagNumArticles_{lag}'] = df['NumArticles'].shift(lag)
        # df[f'lagNumMentions_{lag}'] = df['NumMentions'].shift(lag)
        # df[f'lagQuadClass_{lag}'] = df['QuadClass'].shift(lag)
#     drop columns that refer to same day data
    
    columns_to_drop = ['GoldsteinScale', 'AvgTone', 'NumArticles', 'NumMentions', 'QuadClass', 'lag_fut_Contains14 1']
    df_removed=df_lag.copy()
    df_removed = df_removed.drop(columns_to_drop, axis=1)
    df = df_removed.copy() 
    # roll_range = [1]  # Specify the desired lookback periods

    roll_mean_features = pd.DataFrame()
    roll_mean_features['will_be_1_anytime_next_week_lag'] = df_removed[target_name].shift(min(lookback_lag_target))
    roll_mean_features = [roll_mean_features['will_be_1_anytime_next_week_lag'].rolling(roll).mean() for roll in roll_range]
    roll_mean_columns = [f'roll_mean_{roll}' for roll in roll_range]
    roll_mean_df = pd.concat(roll_mean_features, axis=1)
    roll_mean_df.columns = roll_mean_columns

    df = pd.concat([df, roll_mean_df], axis=1)
    
#      # Compute EMA
#     ema_features = pd.DataFrame()
#     ema_features['will_be_1_anytime_next_week_lag'] = df_removed[target_name].shift(min(lookback_lag_target))
#     ema_features = [ema_features['will_be_1_anytime_next_week_lag'].ewm(span=roll).mean() for roll in roll_range]
#     ema_columns = [f'ema_{roll}' for roll in roll_range]
#     ema_df = pd.concat(ema_features, axis=1)
#     ema_df.columns = ema_columns
    
#     df = pd.concat([df, ema_df], axis=1)
    
    # Add Fourier terms
#     t = np.arange(len(df))
#     frequencies = {
#         'bi_weekly': 1/14,
#         'weekly': 1/7,
#         # 'monthly': 1/30,
#         # 'yearly': 1/365
#     }
    
#     for cycle, freq in frequencies.items():
#         df[f'Fourier_{cycle}_sin'] = np.sin(2 * np.pi * freq * t)
#         df[f'Fourier_{cycle}_cos'] = np.cos(2 * np.pi * freq * t)

    return df


lookback_lag_target = range(window_gap+1,lookback+1)
roll_range = range(window_gap+1,lookback-1)
data_temp=data.copy()
data_process = rolling(data_temp, lookback_lag_target, roll_range)
data_process.dropna(inplace=True)

In [21]:
data_process.head()

,SQLDATE,Year,Month,DayOfMonth,will_be_1_anytime_next_2_days,lag_1,lag_2,lag_3,lag_4,lag_5,...,lag_9,lag_10,roll_mean_1,roll_mean_2,roll_mean_3,roll_mean_4,roll_mean_5,roll_mean_6,roll_mean_7,roll_mean_8
10,2015-03-01,2015,3,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2015-03-02,2015,3,2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,2015-03-03,2015,3,3,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,2015-03-04,2015,3,4,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,2015-03-05,2015,3,5,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def create_train_test_sequences_ordered(data, seq_length=20, target_column=None, test_size=0.2):
    """
    Create training and testing sequences from the data while maintaining the temporal order.
    
    Parameters:
    - data: DataFrame containing the data
    - seq_length: Length of the sequences
    - target_column: Name of the target column
    - test_size: Proportion of the dataset to include in the test split
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    
    if not target_column:
        raise ValueError("Target column must be specified.")
    
    # Get all features except target column
    features = [col for col in data.columns if col != target_column]
    
    num_data = len(data) - seq_length + 1
    num_features = len(features)
    
    X, y = [], []
    
    for i in range(num_data):
        seq = data[features].iloc[i:i+seq_length].values
        target = data[target_column].iloc[i+seq_length-1]
        
        X.append(seq)
        y.append(target)
        
#     split_index = int(len(X) * (1 - test_size))
    
#     X_train, X_test = X[:split_index], X[split_index:]
#     y_train, y_test = y[:split_index], y[split_index:]
    
    return np.array(X), np.array(y)


# Get training and testing sequences while maintaining the temporal order
X, y = create_train_test_sequences_ordered(data_process, seq_length=20, target_column=target_name)
X.shape, y.shape


((1747, 20, 22), (1747,))

In [26]:
# Determine the split index
train_size = int(0.8 * len(X))

# Split the data
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((1397, 20, 22), (350, 20, 22), (1397,), (350,))

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim

# Check if CUDA is available and set our device to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to PyTorch tensors (assuming X_train, y_train, X_test, y_test are already defined)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Assuming the sequence length is equal to the number of time steps in each sample
seq_length = X_train_tensor.shape[1]

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

## LSTM

In [ ]:


class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=1):
        super(LSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # Initialize hidden state and cell state
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device))

    def forward(self, x):
        # Forward pass through LSTM layer
        # lstm_out, self.hidden = self.lstm(x.view(len(x), seq_length, -1))
        lstm_out, self.hidden = self.lstm(x.view(x.shape[0], seq_length, -1))
        
        # Only take the output from the final timestep
        y_pred = self.linear(lstm_out[:, -1, :])
        return y_pred

# Define hyperparameters
input_dim = X_train_tensor.shape[2]  # number of features
hidden_dim = 256
batch_size = 32  # we will use batch size of 1 for simplicity
output_dim = 1
num_layers = 2

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_dim, hidden_dim, batch_size, output_dim, num_layers)
model = model.to(device)
imbalance_ratio = 8

criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([imbalance_ratio]).to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)




In [17]:
torch.onnx.export(model, X_train_tensor, 'iris.onnx', input_names=["features"], output_names=["output"])

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## CNN-LSTM

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ConvLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=1):
        super(ConvLSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the CNN layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.1)

        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.1)
        
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.batch_norm3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.1)

        self.pool = nn.MaxPool1d(kernel_size=2)

        # Define the LSTM layer
        # self.lstm = nn.LSTM(128, self.hidden_dim, self.num_layers)
        self.lstm = nn.LSTM(256, self.hidden_dim, self.num_layers)
        # self.lstm = nn.LSTM(64, self.hidden_dim, self.num_layers, dropout=0.2 if num_layers > 1 else 0)
        # self.lstm = nn.LSTM(64, self.hidden_dim, self.num_layers)
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # Initialize hidden state and cell state
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device))

    def forward(self, x):
        # Forward pass through CNN layers
        x = self.conv1(x)
        # x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.pool(x)

        x = self.conv2(x)
        # x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        # x = self.batch_norm3(x)
        x = self.relu(x)
        x = self.dropout3(x)
        x = self.pool(x)
        

        # Calculate new sequence length for LSTM
        seq_length = x.shape[2]

        # Forward pass through LSTM layer
        # lstm_out, self.hidden = self.lstm(x.view(len(x), seq_length, -1))
        lstm_out, self.hidden = self.lstm(x.view(x.shape[0], seq_length, -1))

        # Only take the output from the final timestep
        y_pred = self.linear(lstm_out[:, -1, :])
        return y_pred

# Define hyperparameters
input_dim = X_train_tensor.shape[1]  # number of channels
hidden_dim = 256
batch_size = 32
output_dim = 1
num_layers = 2

# Initialize the model, loss function, and optimizer
model = ConvLSTMModel(input_dim, hidden_dim, batch_size, output_dim, num_layers)
model = model.to(device)
imbalance_ratio = 8

criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([imbalance_ratio]).to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [21]:
torch.onnx.export(model, X_train_tensor, 'iris.onnx', input_names=["features"], output_names=["output"])

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## BIDIRECTIONAL

In [118]:
class BidirectLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1, num_layers=1):
        super(ConvLSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the CNN layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=7, padding=1)
        self.batch_norm1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.1)
        
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=7, padding=1)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.1)
        
        self.pool = nn.MaxPool1d(kernel_size=2)

        # Define the LSTM layer with bidirectionality
        self.lstm = nn.LSTM(128, self.hidden_dim, self.num_layers, 
                            dropout=0.2 if num_layers > 1 else 0, bidirectional=True)

        # Adjust the input size for the linear layer to account for bidirectionality
        self.linear = nn.Linear(2 * self.hidden_dim, output_dim)

    def init_hidden(self):
        # Adjust the initialization for bidirectional hidden states
        return (torch.zeros(2 * self.num_layers, self.batch_size, self.hidden_dim).to(device),
                torch.zeros(2 * self.num_layers, self.batch_size, self.hidden_dim).to(device))

    def forward(self, x):
        # Forward pass through CNN layers
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.pool(x)

        # Forward pass through LSTM layer
        lstm_out, self.hidden = self.lstm(x.view(len(x), seq_length, -1))
        
        # Only take the output from the final timestep
        # Remember to concatenate the outputs from both directions
        y_pred = self.linear(torch.cat((lstm_out[:, -1, :self.hidden_dim], 
                                        lstm_out[:, 0, self.hidden_dim:]), 1))
        return y_pred

        # Define hyperparameters
input_dim = X_train_tensor.shape[2]  # number of features
hidden_dim = 256
batch_size = 32  # we will use batch size of 1 for simplicity
output_dim = 1
num_layers = 2

# Initialize the model, loss function, and optimizer
model = BidirectLSTMModel(input_dim, hidden_dim, batch_size, output_dim, num_layers)
model = model.to(device)
imbalance_ratio = 8

criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([imbalance_ratio]).to(device))
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Training and test

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix
def compute_f1(model, X, y_true):
    with torch.no_grad():
        y_pred = model(X)
        y_pred_binary = (torch.sigmoid(y_pred).cpu().numpy() > 0.5).astype(int)
        cm = confusion_matrix(y_true, y_pred_binary)
    return f1_score(y_true, y_pred_binary), cm

# Retrain the model and compute f1 scores every 10 epochs
num_epochs = 100
f1_train_scores = []
f1_test_scores = []

for epoch in range(num_epochs):
    model.train()
    model.hidden = model.init_hidden()
    optimizer.zero_grad()
    
    # Forward pass
    y_pred = model(X_train_tensor)
    
    # Calculate loss
    loss = criterion(y_pred.view(-1), y_train_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Compute f1 scores every 10 epochs
    if (epoch) % 20 == 0:
        f1_train, _ = compute_f1(model, X_train_tensor, y_train)
        f1_test, cm = compute_f1(model, X_test_tensor, y_test)
        print(epoch, f1_test, f1_train, cm)
        f1_train_scores.append(f1_train)
        f1_test_scores.append(f1_test)
        


Permutation Importance for LSTM:
The steps are as follows:

Record the performance of the model with the original data (e.g., F1 score).

For each feature:
a. Permute the values of this feature across all samples.
b. Record the performance of the model with this permuted data.
c. The importance of this feature is the difference between the original performance and the permuted performance.

Rank features based on their importance.

In [ ]:
def averaged_permutation_importance(model, X, y_true, metric, n_repeats=30):
    """
    Compute averaged permutation importance for features in X using multiple permutations.
    
    Parameters:
    - model: trained LSTM model
    - X: input data (tensor)
    - y_true: true labels
    - metric: function to compute performance metric
    - n_repeats: number of times to repeat the permutation process for each feature
    
    Returns:
    - importances: array of averaged importances for each feature
    """
    # Convert tensors to numpy for permutation
    X_np = X.cpu().numpy()
    y_true_np = y_true.cpu().numpy()
    
    # Compute original metric
    original_score, _ = metric(model, X, y_true_np)
    
    # Placeholder for importances
    importances = np.zeros(X_np.shape[2])
    
    # Compute permutation importance
    for i in range(X_np.shape[2]):
        importance_accumulator = 0
        for _ in range(n_repeats):
            X_permuted = X_np.copy()
            np.random.shuffle(X_permuted[:,:,i])
            X_permuted_tensor = torch.tensor(X_permuted).to(device)
            score_permuted, _ = metric(model, X_permuted_tensor, y_true_np)
            importance_accumulator += original_score - score_permuted
        # Average the accumulated importances
        importances[i] = importance_accumulator / n_repeats
    
    return importances

# Assuming target_name is defined and is the name of the target variable in data_process
# Exclude target_name from the DataFrame to get feature names
feature_names = data_process.drop(columns=[target_name]).columns.tolist()

# Compute averaged permutation importances for the trained model using the test data
avg_importances = averaged_permutation_importance(model, X_test_tensor, y_test_tensor, compute_f1)

# Plot the averaged importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(avg_importances)), avg_importances)
plt.xlabel('Features')
plt.xticks(range(len(avg_importances)), feature_names, rotation=45)
plt.ylabel('Importance')
plt.title('Averaged Permutation Importance of LSTM Features')
plt.show()
